<a href="https://colab.research.google.com/github/yuukun123/Python/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

# Tùy chỉnh để hiển thị biểu đồ đẹp hơn
sns.set(style="whitegrid")

Mounted at /content/drive


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/Final_Cleaned_Database.xlsx")
print(df.to_string())

In [4]:
# Đường dẫn file
file_path = "/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/DelawarePainDatabase_SecondaryDatabase_FaceGenCharacterization.xlsx"
xls = pd.ExcelFile(file_path)

hàm lọc dữ liệu sheet ExpressionAverage


In [ ]:
# def clean_expression_sheet(df, required_cols=["Expression"]):
#     # Xóa cột không tên
#     df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

#     # Xóa dòng thiếu cột bắt buộc
#     df = df.dropna(subset=required_cols)

#     # Loại bỏ các biểu cảm không dùng
#     df = df[~df["Expression"].str.lower().str.contains("pms13")]
#     df = df[~df["Expression"].str.lower().str.contains("facegenpro_avgblackface")]

#     # Các cột cần ép kiểu và xử lý NaN
#     numeric_cols = ["ThreatRating", "ConfusionRating", "ThreatComp", "ConfusionComp"]
#     for col in numeric_cols:
#         if col in df.columns:
#             df[col] = df[col].replace([".", ""], np.nan)
#             df[col] = pd.to_numeric(df[col], errors="coerce")
#             df[col] = df[col].fillna(df[col].mean())

#     # Loại bỏ các dòng liên quan face gen pro
#     df = df[~df["Norming Survey"].str.lower().str.contains("facegenpro")]

#     df = df.infer_objects(copy=False)

#     return df


In [ ]:
def clean_expression_sheet(df, required_cols=["Expression"]):
    # Xóa cột không tên
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # Xóa dòng thiếu cột bắt buộc
    df = df.dropna(subset=required_cols)

    # Loại bỏ các biểu cảm không dùng
    df = df[~df["Expression"].str.lower().str.contains("pms13")]
    df = df[~df["Expression"].str.lower().str.contains("facegenpro_avgblackface")]

    # ✅ Loại bỏ các dòng liên quan face gen pro TRƯỚC để không ảnh hưởng đến mean
    df = df[~df["Norming Survey"].str.lower().str.contains("facegenpro")]

    # Các cột cần ép kiểu và xử lý NaN
    numeric_cols = ["ThreatRating", "ConfusionRating", "ThreatComp", "ConfusionComp"]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].replace([".", ""], np.nan)
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df[col] = df[col].fillna(df[col].mean())

    df = df.loc[:, ~df.columns.str.contains("ConfusionRating")]
    df = df.loc[:, ~df.columns.str.contains("ConfusionComp")]

    df = df.infer_objects(copy=False)
    return df


ExpressionAverages

In [ ]:
df_expr = pd.read_excel(xls, sheet_name="ExpressionAverages")
# print(df_expr.dtypes)
df_expr = clean_expression_sheet(df_expr)

print(df_expr.to_string())

hàm lọc dữ liệu sheet slider_face_gen

In [ ]:
def clean_slider_facegen_sheet(df):
    # Xóa các cột không tên (thường là chú thích hoặc rỗng)
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # Loại bỏ cột 'PMS13' nếu có (theo khuyến nghị)
    if "PMS13" in df.columns:
        df = df.drop(columns=["PMS13"])

    # Duyệt qua tất cả các cột trừ 'Slider'
    numeric_cols = df.columns.drop("Slider")
    for col in numeric_cols:
        # Thay "." và chuỗi rỗng thành NaN
        df[col] = df[col].replace([".", ""], np.nan)

        # Ép kiểu thành số (sai sẽ thành NaN)
        df[col] = pd.to_numeric(df[col], errors="coerce")

        # Điền thiếu bằng trung bình cột
        df[col] = df[col].fillna(df[col].mean())

    # Bỏ cảnh báo downcasting
    df = df.infer_objects(copy=False)

    return df


Slider_face_gen

In [ ]:
df_slider_mo = pd.read_excel(xls, sheet_name="Sliders_FaceGenModeller")
# print(df_slider_mo.dtypes)
df_slider_mo = clean_slider_facegen_sheet(df_slider_mo)
print(df_slider_mo.to_string())

hàm lọc first_pass_norming

In [ ]:
# def clean_first_pass_norming(df):
#     import pandas as pd


#     # 1. Loại cột chứa 'pms13'
#     df = df.loc[:, ~df.columns.str.contains("pms13", case=False)]

#     # 2. Loại bỏ các cột nhân khẩu học không cần thiết
#     drop_cols = ['Race', 'SexualOrient', 'Political']
#     df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')

#     # 3. Tìm các cột đánh giá cảm xúc
#     rating_cols = [c for c in df.columns if "emotion" in c.lower()]

#     # 4. Tính tỷ lệ thiếu
#     miss_ratios = df[rating_cols].isna().mean()

#     # 5. Giữ lại các cột có missing <= 0.4
#     keep_cols = miss_ratios[miss_ratios <= 0.4].index.tolist()
#     df = df[keep_cols + [col for col in df.columns if col not in rating_cols]]

#     # 6. Phân loại cột theo mức độ thiếu
#     small_missing = miss_ratios[miss_ratios < 0.05].index.tolist()
#     large_missing = miss_ratios[(miss_ratios >= 0.05) & (miss_ratios <= 0.4)].index.tolist()

#     # 7. Drop dòng chứa missing < 5%
#     df = df.dropna(subset=small_missing)

#     # 8. Điền missing >= 5% bằng mode
#     for col in large_missing:
#         mode_val = df[col].mode().iat[0]
#         df[col] = df[col].fillna(mode_val).astype(int)

#     # 9. Reshape wide → long format
#     id_vars = ['ResponseId', 'Age', 'Gender']
#     id_vars = [col for col in id_vars if col in df.columns]

#     long_df = df.melt(
#         id_vars=id_vars,
#         value_vars=keep_cols,
#         var_name="Item",
#         value_name="Rating"
#     )

#     # 10. Tách thông tin từ tên cột: ImageID, EmotionType, QuestionNumber
#     long_df[['ImageID', 'EmotionType', 'QuestionNum']] = long_df['Item'].str.extract(r'([A-Za-z0-9_]+)_([a-zA-Z0-9]+)_([0-9]+)')

#     # 11. Xử lý kiểu dữ liệu
#     long_df['Rating'] = pd.to_numeric(long_df['Rating'], errors='coerce')
#     long_df['QuestionNum'] = pd.to_numeric(long_df['QuestionNum'], errors='coerce')

#     return long_df


In [6]:
def clean_first_pass_norming(df):
    # 1. Loại cột chứa 'pms13'
    df = df.loc[:, ~df.columns.str.contains("pms13", case=False)]



Fist_pass_norming

In [10]:
df_first = pd.read_excel(xls, sheet_name="FirstPassNorming")
# df_first = clean_first_pass_norming(df_first)


# print(df_first.to_string())
# df_first.info()
df_first.head()


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,StartDate,ResponseId,EF1_emotion_18,EF1_emotion_19,EF1_emotion_20,EF1_emotion_21,EF1_emotion_22,EF1_emotion_23,EF1_emotion_24,EF1_emotion_25,...,KM4b_emotion_22,KM4b_emotion_23,KM4b_emotion_24,KM4b_emotion_25,Age,Gender,SexualOrient,Race,Race_7_TEXT,Political
0,Start Date,Response ID,"Now, consider the person pictured below and ra...","Now, consider the person pictured below and ra...","Now, consider the person pictured below and ra...","Now, consider the person pictured below and ra...","Now, consider the person pictured below and ra...","Now, consider the person pictured below and ra...","Now, consider the person pictured below and ra...","Now, consider the person pictured below and ra...",...,"Now, consider the person pictured below and ra...","Now, consider the person pictured below and ra...","Now, consider the person pictured below and ra...","Now, consider the person pictured below and ra...",Age,Gender,Which of the following categories best fits yo...,What is your race? - Selected Choice,What is your race? - Other - Text,Ideology
1,2017-04-28 17:33:04,R_2uEXmS5nNEIe2xg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,3,4,3,25,female,Straight,Native American,NaN,Very Liberal
2,2017-04-28 17:32:37,R_2zw5psM5CtFQKno,2,2,3,2,6,3,2,1,...,2,2,2,6,26,male,Straight,White/Caucasian,NaN,Somewhat liberal
3,2017-04-28 17:33:58,R_3CIA7Qxhba1ElBs,5,6,3,1,1,1,5,6,...,3,1,2,7,49,Female,Straight,White/Caucasian,NaN,Liberal
4,2017-04-28 17:35:32,R_2Ua2Dc353v4cNGn,1,2,2,1,2,2,2,2,...,4,2,2,6,32,female,Straight,White/Caucasian,NaN,Conservative


hàm lọc demographics

In [ ]:
def clean_demographics(df):

  cols_to_drop = df.columns[6:20]

  # Drop những cột đó
  df = df.drop(columns=cols_to_drop)

  # Xoá các cột đã chỉ định nếu tồn tại
  df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

  # Giữ lại 'Unnamed: 1', xóa các cột 'Unnamed' khác
  cols_to_drop = [col for col in df.columns if 'Unnamed' in col and col != 'Unnamed: 1']
  df = df.drop(columns=cols_to_drop)

  # # Đổi tên cột nếu cần, ví dụ: đổi 'Unnamed: 1' thành 'Gender'
  # df = df.rename(columns={'Unnamed: 1': 'Gender'})

  # Xoá các tên cột chứa 'Unnamed' còn sót lại (nếu có)
  df.columns = [col if 'Unnamed' not in col else '' for col in df.columns]

  return df

Demographics

In [ ]:
df_demographics = pd.read_excel(xls, sheet_name="Demographics")
df_demographics = clean_demographics(df_demographics)
print(df_demographics.to_string())


   FIRST PASS         
0         Age   Gender
1          25   Female
2          26     Male
3          49  Female 
4          32   Female
5          64   Female
6          36   Female
7          49     Male
8          32   Female
9          25     Male
10         24     Male
11         45   Female
12         32   Female
13         21     Male
14         39     Male
15         59   Female
16         31   Female
17         39     Male
18         31     Male
19         39     Male
20         48   Female
21         25     Male
22         40   Female
23         24     Male
24         40   Female
25         27     Male
26         36     Male
27         26   Female
28         35     Male
29         36     Male
30         26     Male
31         37     Male
32         61     Male
33         48     Male
34         31   Female
35         36   Female
36         33   Female
37         21   Female
38         46     Male
39         46     Male
40         40     Male
41         28   Female
42         

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
# Giả sử 4 DataFrame đã tồn tại: df_expr, df_slider_mo, df_first, df_demographics

# Ghi vào file Excel với 4 sheet
with pd.ExcelWriter('/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/output_cleaned.xlsx') as writer:
    df_expr.to_excel(writer, sheet_name='Expression', index=False)
    df_slider_mo.to_excel(writer, sheet_name='Slider_MO', index=False)
    df_first.to_excel(writer, sheet_name='First', index=False)
    df_demographics.to_excel(writer, sheet_name='Demographics', index=False)
